# pytorch - Autograd
autograd는 신경망 학습을 지원하는 pytorch의 자동 미분 엔진이다.

In [1]:
import torch 

In [2]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print("Current cuda decive: ", torch.cuda.current_device())

Current cuda decive:  1


In [3]:
batch_size = 64  # 파라미터를 업데이트 할 때 계산되는 데이터의 개수
input_size = 1000
hidden_size = 100
output_size = 10

In [4]:
x = torch.randn(batch_size, input_size, device=device, dtype = torch.float, requires_grad = False)

- randn : 평균이 0, 표준편차가 1인 정규분포에서 샘플링한 값으로, 데이터를 만든다는 것을 의미. batch_size, input_size에 따라 데이터의 모양 달라짐.  
- x는 input으로 이용되기 때문에 gradient 계산할 필요 없다. 따라서 requires_grade = False  


In [5]:
y = torch.randn(batch_size, output_size, device=device, dtype=torch.float, requires_grad=False) 

- output 설정. batch_size 만큼 결과값 필요.
- output과의 오차를 계산하기 위해 output의 크기를 10으로 설정

In [6]:
w1 = torch.randn(input_size, hidden_size, device = device, dtype = torch.float,requires_grad = True)  

- 업데이트 할 파라미터 값 설정
- input data 크기 1000이며 이것과 행렬곱을 하므로 다음 행의 값이 1000이어야 함.
- 행렬곱을 이용해 100 크기의 데이터를 생성하기 위해 (1000, 100)크기의 데이터를 생성.
- x, y와 다르게 gradient를 계산해야 하므로 requires_grad = True로 설정

In [7]:
w2 = torch.randn(hidden_size,output_size, device = device,dtype = torch.float,requires_grad = True) 

- w1과 x를 행렬 곱한 결과에 계산할 수 있는 데이터여야 한다.
- w1과 x의 행렬 곲을 한 결과는 (1,100)이며 (100, 10)행렬을 통해 output을 계산할 수 있도록 한다.
- backpropagation을 통해 업데이트해야하는 대상이므로 requires_grad = True로 설정

In [8]:
learning_rate = 1e-6     
for i in range(1, 501):
    y_pred = x.mm(w1).clamp(min = 0).mm(w2)
    
    loss = (y_pred - y).pow(2).sum()  # 제곱차의 합
    if i % 100 == 0:
        print("Iteration: ", i, "\t", "Loss: ", loss.item()) 
        
    loss.backward()

    with torch.no_grad():                                      
        w1 -= learning_rate * w1.grad                          
        w2 -= learning_rate * w2.grad                          

        w1.grad.zero_()                                      
        w2.grad.zero_()

Iteration:  100 	 Loss:  309.1791687011719
Iteration:  200 	 Loss:  1.4110450744628906
Iteration:  300 	 Loss:  0.011791206896305084
Iteration:  400 	 Loss:  0.0002832590544130653
Iteration:  500 	 Loss:  4.207486199447885e-05


- clamp : 값의 상한선/하한선 설정 (min=0이므로 0보다 작은 값이 발생하면 그 값을 0으로 바꿔줌 = ReLU와 같은 역할.)
- pow : 지수를 의미. (2)이므로 제곱.
- backward : 계산된 loss 값에 대해 사용하면 각 파라미터 값에 대해 gradient를 계산하고 이를 통해 back propagation을 진행한다는 것을 의미.
- with절 : 코드가 실행되는 시점에서 gradient값을 고정한다는 의미
- grad.zero() : gradient값을 0으로 설정. 다음 backpropagation을 진행할 때 gradient 값을 loss.backward()를 통해 새로 계산하기 때문